### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 以句點和逗點為分隔拆句子

In [2]:
import json
import nltk

In [3]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [4]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [5]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    sentences_2 = []
    for s in sentences:
        sentences_2.extend(s.split("，"))
        
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
    for sent in sentences_2:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

188347


In [7]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 188347
print(ave)

7.707476094655078


In [8]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [9]:
print(len(data))

188347


In [10]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [11]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
print(len(X_train))
print(len(X_test))

150677
37670


In [13]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 30160
Seconds required: 2.707

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 290837.797120
Feature norm: 1.000000
Error norm: 123948.319448
Active features: 30044
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 1.007

***** Iteration #2 *****
Loss: 286017.046950
Feature norm: 1.031950
Error norm: 97854.047695
Active features: 28371
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.507

***** Iteration #3 *****
Loss: 273137.250274
Feature norm: 1.147125
Error norm: 70783.721712
Active features: 25876
Line search trials: 1
Line search step: 1.000000
Seconds required

***** Iteration #43 *****
Loss: 48140.177395
Feature norm: 30.573278
Error norm: 29387.579079
Active features: 26110
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.512

***** Iteration #44 *****
Loss: 47196.313441
Feature norm: 31.953126
Error norm: 11752.474617
Active features: 25998
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.509

***** Iteration #45 *****
Loss: 46277.598035
Feature norm: 33.323614
Error norm: 29468.228266
Active features: 25856
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.505

***** Iteration #46 *****
Loss: 45221.402876
Feature norm: 34.203873
Error norm: 4816.981651
Active features: 25782
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.528

***** Iteration #47 *****
Loss: 44213.249840
Feature norm: 35.079674
Error norm: 5087.493667
Active features: 25697
Line search trials: 1
Line search step: 1.000

***** Iteration #88 *****
Loss: 34081.184607
Feature norm: 93.074522
Error norm: 2947.495459
Active features: 21977
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.508

***** Iteration #89 *****
Loss: 34048.561918
Feature norm: 93.628310
Error norm: 2856.991546
Active features: 21941
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.508

***** Iteration #90 *****
Loss: 34007.838818
Feature norm: 94.278733
Error norm: 4123.789187
Active features: 21771
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.507

***** Iteration #91 *****
Loss: 33981.586759
Feature norm: 95.172807
Error norm: 6655.453734
Active features: 21775
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.506

***** Iteration #92 *****
Loss: 33945.354705
Feature norm: 95.606851
Error norm: 2684.441218
Active features: 21851
Line search trials: 1
Line search step: 1.000000

***** Iteration #128 *****
Loss: 33369.847849
Feature norm: 108.768999
Error norm: 3230.195248
Active features: 21254
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.505

***** Iteration #129 *****
Loss: 33361.786273
Feature norm: 109.046487
Error norm: 2391.431914
Active features: 21240
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.508

***** Iteration #130 *****
Loss: 33354.349269
Feature norm: 109.199567
Error norm: 1993.531148
Active features: 21240
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.505

***** Iteration #131 *****
Loss: 33346.996373
Feature norm: 109.445995
Error norm: 2173.173413
Active features: 21208
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.509

***** Iteration #132 *****
Loss: 33339.285621
Feature norm: 109.665655
Error norm: 3140.582386
Active features: 21200
Line search trials: 1
Line search step

***** Iteration #168 *****
Loss: 33118.119513
Feature norm: 115.184189
Error norm: 3164.807631
Active features: 20820
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.510

***** Iteration #169 *****
Loss: 33111.981441
Feature norm: 115.341080
Error norm: 2106.113850
Active features: 20818
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.506

***** Iteration #170 *****
Loss: 33107.248396
Feature norm: 115.418000
Error norm: 1801.043745
Active features: 20810
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.505

***** Iteration #171 *****
Loss: 33102.953654
Feature norm: 115.536116
Error norm: 1775.876724
Active features: 20807
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.507

***** Iteration #172 *****
Loss: 33099.026691
Feature norm: 115.653092
Error norm: 2302.023542
Active features: 20800
Line search trials: 1
Line search step

In [14]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

但 (O)
表 (O)
热 (O)
甚 (O)


In [15]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-S","I-S","E-S"]))

             precision    recall  f1-score   support

          O       0.97      0.98      0.98    277607
        B-S       0.61      0.47      0.53      4496
        I-S       0.45      0.32      0.38      4856
        E-S       0.62      0.47      0.53      4476

avg / total       0.95      0.96      0.95    291435

